# Books Scraping Project
* Scrap the books (name, price, rate, category_name) for each category and put them into a CSV & Excel file
* https://books.toscrape.com/

#### Import some helper modules

In [4]:
import os
import time
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import re
import csv 
import json
import openpyxl
import sqlite3

#### Get category name and it's url 
* cat_link is a dict has category {name : url} 

In [5]:
page_url = 'https://books.toscrape.com/'
req = requests.get('https://books.toscrape.com/')
soup = BeautifulSoup(req.content)
body = soup.body
cat_list = body.find('ul' , {'class':"nav nav-list"}).find('ul').find_all('li')
cat_link = {li.find('a').text.strip():page_url+li.find('a').attrs['href'] for li in cat_list}
cat_link['Travel']

'https://books.toscrape.com/catalogue/category/books/travel_2/index.html'

#### Save Data in Database
* save categories in categories table 
* categories table contains 2 coloumns (cat_id -- > category id , cat_name -- > category name)

In [3]:
conn = sqlite3.connect('BooksInfo.db')
qry = 'insert into categories (cat_name) values (?)'
for k , val in cat_link.items():
    conn.execute(qry , [k])
conn.commit()
conn.close()

#### Scrape Process
* scrape function take url as input
* return (name , rate , price) for all books in any category
* if there are another pages in category return link of next page 

In [6]:
def scrape (url):
    trans_rate = {'One' : '1' , 'Two' : '2' , 'Three' : '3' , 'Four' : '4' , 'Five' : '5'}
    req = requests.get(url)
    soup = BeautifulSoup(req.content , 'html.parser')
    body = soup.body
    books_info = body.find_all('li',{'class':"col-xs-6 col-sm-4 col-md-3 col-lg-3"})
    names = [book.find('div' , {'class' : "image_container"}).find('img').attrs['alt'] 
             for book in books_info]
    rates = [book.find('p' , {'class' : 'star-rating'}).attrs['class'][1] 
             for book in books_info]
    encoded_rates = [trans_rate[rate] for rate in rates]
    price = [book.find('p' , {'class':"price_color"}).text.split('£')[1] 
             for book in books_info]
    next_bottun = body.find('li' , {'class':"next"})
    anchor = next_bottun.find('a').attrs['href'] if next_bottun != None else ''
    return names , encoded_rates , price , anchor
#scrape(cat_link['Mystery'])

#### save books info in CSV file
* write category name , book name , rate , price 

In [7]:
def write_csv (cat ,names , encoded_rates , price):
    fn = ['Category' ,'Name' , 'Rate' , 'Price']
    m = 'w' if not os.path.exists('Books.csv') else 'a'
    with open('Books.csv' , m , newline='') as f :
        writer = csv.DictWriter(f , fieldnames=fn)
        if m == 'w':
            writer.writeheader()
        for nam , rat , pric in zip(names , encoded_rates , price):
            writer.writerow({'Category':cat ,'Name' : nam , 'Rate' : rat , 'Price' : pric})

#### Save books info into excel file 
* write category name , book name , rate , price 

In [8]:
def write_xl(cat ,names , encoded_rates , price):
    fn = ['Category' ,'Name' , 'Rate' , 'Price']
    if not os.path.exists('Books.xlsx'):
        wb = openpyxl.Workbook()
        sheet = wb.active
        row , col = sheet.max_row , sheet.max_column
        for i in fn:
            sheet.cell(row , col ).value = i
            col += 1
        wb.save('Books.xlsx')
    wb = openpyxl.load_workbook('Books.xlsx')
    sheet = wb.active
    row = sheet.max_row + 1
    for nam , rat , pric in zip(names , encoded_rates , price):
        sheet.cell(row , 1).value = cat
        sheet.cell(row , 2).value = nam
        sheet.cell(row , 3).value = rat
        sheet.cell(row , 4).value = pric
        row += 1
    wb.save('Books.xlsx')

#### Save books info into database
* books table contain 3 coloumns 
* book id , name , rate , price and cat_id as foriegn key from table categories

In [26]:
def write_database(cat ,names , encoded_rates , price):
    qry = 'select cat_id from categories where cat_name = (?)'
    conn = sqlite3.connect('BooksInfo.db')
    cat_id = conn.execute(qry , [cat]).fetchall()[0][0]
    qry = 'insert into books (book_name , book_price , book_rate , cat_id) values(?,?,?,?)'
    for nam , pric , rat in zip(names , price , encoded_rates):
        conn.execute(qry , [nam , pric , rat , cat_id])
    conn.commit()
    conn.close()

#### Scraping Engine 
* Iterate on each category and scrape books info 
* if there are anthor pages get the next bage url to scrape it 
* then save books info in CSV & excel files 

In [28]:
for cat , url in cat_link.items():
    names , encoded_rates , price , anchor = scrape(url)
    if anchor == '':
        #write_csv(cat ,names , encoded_rates , price)
        #write_xl(cat ,names , encoded_rates , price)
        write_database(cat ,names , encoded_rates , price)
    else:
        #print(cat, '  ',anchor)
        #write_csv(cat ,names , encoded_rates , price)
        #write_xl(cat ,names , encoded_rates , price)
        write_database(cat ,names , encoded_rates , price)
        while anchor != '':
            print(cat, '  ',anchor)
            new_url = url.replace('index',anchor.split('.')[0])
            print(new_url)
            names , encoded_rates , price , anchor = scrape(new_url)
            #write_csv(cat ,names , encoded_rates , price)
            #write_xl(cat ,names , encoded_rates , price)
            write_database(cat ,names , encoded_rates , price)

Mystery    page-2.html
https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
Historical Fiction    page-2.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html
Sequential Art    page-2.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html
Sequential Art    page-3.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html
Sequential Art    page-4.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html
Romance    page-2.html
https://books.toscrape.com/catalogue/category/books/romance_8/page-2.html
Fiction    page-2.html
https://books.toscrape.com/catalogue/category/books/fiction_10/page-2.html
Fiction    page-3.html
https://books.toscrape.com/catalogue/category/books/fiction_10/page-3.html
Fiction    page-4.html
https://books.toscrape.com/catalogue/category/books/fiction_10/page-4.html
Childrens    page-2.html
https://books.toscrape.com/catalogu

In [3]:
#os.remove('Books.csv')
#os.remove('Books.xlsx')

In [54]:
#!Books.csv
#!Books.xlsx